Assessing the stability of the model:


Calculate the pathways that vary the most in our output

In [12]:
import analyze
from analyze import Posterior
%load_ext autoreload
%autoreload 2



posteriors = [Posterior('results/' + str(i) + '.csv') for i in range(8, 10)] 
#posteriors = [Posterior('out1.csv'), Posterior('out2.csv')] 
all_paths, top_ranges = analyze.compare_pathway_probs(posteriors, threshold = 0.1)  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
86 86 1478
86 86 1534
top ranges
p_cge00590 0.9817602710419999
p_cge00750 0.9544584847290001
p_cge00770 0.8185167616249
p_cge00240 0.808898324743
p_cge00380 0.694555010615
p_cge00052 0.6913681041119999
p_cge00790 0.6349242603535
p_cge00600 0.634319495934
p_cge00120 0.624960777615
p_cge00250 0.6177040648409999
p_cge00400 0.6088626930006
p_cge00030 0.5954846723493
p_cge00340 0.5675747845525001
p_cge00230 0.565003980597
p_cge01040 0.5625395627709999
p_cge00531 0.5383384012453
p_cge00480 0.537877415651
p_cge00280 0.535886907568
p_cge00670 0.5290780208719
p_cge00630 0.501834034579
p_cge00500 0.49075352921200005
p_cge00970 0.4886392821413
p_cge00310 0.48861353294261
p_cge00471 0.48839191095256
p_cge00565 0.48370903939271004
p_cge00140 0.47946967115620004
p_cge00650 0.4776081113888
p_cge00130 0.47520405129199994
p_cge00604 0.47135488864009994
p_cge00260 0.4647655392194
p_cge00360 0.46036033228300005
p_cge0

The above shows all pathways in our model with a range greater than 0.5 between 4 runs of the sampler.

Lets see what metabolites occur most often in this set of pathways

In [15]:
import structure_analysis as sa
from operator import itemgetter
top_range_set = set(zip(*top_ranges)[0])
DG, pathways, mets = sa.create_metabolite_graph(top_range_set)
degree2mets, degree2edges, common_mets = sa.find_common_mets(DG, mets)      
print ("Common metabolites")
for met, degree in sorted(common_mets, key=itemgetter(1), reverse=True):
    print (met, degree)

ImportError: No module named 'networkx'

Interesting... Lets plot the bipartite graph to see where things might be affecting each other

In [2]:
#import networkx as nx
import matplotlib.pyplot as plt
n_paths, n_mets = len(pathways), len(degree2mets)
pos = dict(zip(sorted(pathways),zip([0]*n_paths, range(n_paths))))
pos.update(dict(zip(sorted(degree2mets), zip([1] * n_mets, range(n_mets)))))
common_G = DG.subgraph(set(degree2mets) | pathways).to_undirected()
nx.draw_networkx(common_G, pos, with_labels=True, node_size=0)
plt.show()

NameError: name 'pathways' is not defined

In [4]:
met = 'C00116'
paths = common_G.neighbors(met)
all_paths, all_mets = analyze.construct_union(posteriors)
for p in paths:
    print p.strip(), all_paths[p]
met, all_mets['g_' + met]


p_cge00052 [0.0350541053432, 0.905563908246, 0.0486663643996, 1.07982521906]
p_cge00561 [0.900269167635, 0.634386867709, 0.0401556916348, 0.627014039379]


('C00116', [])

Weird... The first few mets I tried seem to be observed

In [5]:
metlin, hmdb, metfrag = analyze.get_metabolite_sets()
observed = metlin | hmdb | metfrag
for met in common_mets:
    print met, met[0] in observed

('C00159', 4) True
('C00158', 3) True
('C00122', 5) True
('C00116', 2) True
('C00095', 2) True
('C01179', 3) True
('C00137', 3) True
('C00073', 4) True
('C00135', 4) True
('C00062', 5) True
('C00270', 2) True
('C00109', 2) True
('C00042', 4) True
('C00311', 4) True
('C00794', 2) True
('C00191', 3) True
('C00186', 2) True
('C00267', 5) True
('C03406', 2) True
('C00249', 3) True


This is definitly not a coincidence.

Let's take a look at the compound cge00350
This is sometimes on sometimes off... But should definitly be on.

The problem appears to be C00025 (glutimate)

In [7]:
analyze.summarize_compound('C00025')
analyze.summarize('cge00350')

Compound C00025 has pathways:
cge00130
cge00220
cge00230
cge00240
cge00250
cge00260
cge00270
cge00280
cge00290
cge00300
cge00310
cge00330
cge00340
cge00350
cge00360
cge00380
cge00400
cge00410
cge00460
cge00471
cge00480
cge00520
cge00590
cge00630
cge00640
cge00650
cge00670
cge00750
cge00760
cge00770
cge00790
cge00860
cge00910
cge00970
cge00983
cge01200
cge01210
cge01230
Prob(C05381) 0.75
cge00350 57
Unobserved:
C00025 38
C00122 8
C00026 27
C00021 13
C00022 17
C00483 1
C01829 1
C02515 1
C02514 1
C00272 6
C01161 1
C01382 1
C05422 1
C00788 1
C02218 2
C05581 1
C05580 1
C05583 1
C05582 1
C05585 1
C05584 1
C05587 1
C00822 1
C00218 2
C00642 1
C00268 6
C00011 46
C00014 31
C01693 1
C05588 1
C00072 2
C00019 15
C00355 1
C05350 1
C05578 1
C05579 1
C02465 1
C05589 1
C05576 1
C05577 1
C01179 4
C04185 1
C17938 1
C00082 6
C03758 1
C00080 63
C05590 1
C01036 1
C04043 1
C01060 1
C05594 1
C00164 4
C01061 1
C00628 1
C00547 1
C00544 2
C03765 1
Full Stats:
C00025 38
C01179 4
C00122 8
C00268 6
C00082 6
C00019 

[{'C00025',
  'C00029',
  'C00047',
  'C00049',
  'C00052',
  'C00062',
  'C00064',
  'C00065',
  'C00073',
  'C00078',
  'C00079',
  'C00082',
  'C00089',
  'C00095',
  'C00106',
  'C00114',
  'C00116',
  'C00120',
  'C00123',
  'C00127',
  'C00135',
  'C00137',
  'C00148',
  'C00149',
  'C00152',
  'C00153',
  'C00158',
  'C00159',
  'C00183',
  'C00186',
  'C00188',
  'C00191',
  'C00208',
  'C00212',
  'C00214',
  'C00242',
  'C00243',
  'C00250',
  'C00255',
  'C00257',
  'C00262',
  'C00267',
  'C00270',
  'C00294',
  'C00299',
  'C00307',
  'C00311',
  'C00314',
  'C00328',
  'C00330',
  'C00331',
  'C00352',
  'C00378',
  'C00402',
  'C00407',
  'C00408',
  'C00417',
  'C00427',
  'C00491',
  'C00504',
  'C00526',
  'C00532',
  'C00534',
  'C00547',
  'C00559',
  'C00588',
  'C00637',
  'C00643',
  'C00647',
  'C00695',
  'C00719',
  'C00740',
  'C00763',
  'C00794',
  'C00819',
  'C00847',
  'C00864',
  'C00881',
  'C01083',
  'C01530',
  'C01674',
  'C01697',
  'C01879',
  'C